<a href="https://colab.research.google.com/github/suyash091/EEG-MNIST-Analysis/blob/master/Training_EEG_Ridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import gc
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
Df1=pd.read_csv('/content/drive/My Drive/BCI MNIST/xvar1.csv')
Df2=pd.read_csv('/content/drive/My Drive/BCI MNIST/xvar2.csv')
Df3=pd.read_csv('/content/drive/My Drive/BCI MNIST/xvar3.csv')

In [0]:
bigdata = pd.concat([Df1, Df2, Df3], ignore_index=True, sort =False)
del Df1
del Df2
del Df3

In [6]:
bigdata.drop(bigdata.columns[bigdata.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
bigdata.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3614,3615,3616,3617,3618,3619,3620,3621,3622,3623,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633,3634,3635,3636,3637,3638,3639,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649,3650,3651,3652,3653
0,4395.384615,4382.564102,4377.435897,4387.179487,4395.897435,4398.461538,4396.923076,4391.794871,4391.282051,4395.897435,4398.974358,4396.410256,4392.820512,4392.820512,4395.384615,4398.974358,4394.358974,4381.538461,4376.410256,4383.589743,4387.179487,4388.717948,4397.948717,4401.538461,4397.948717,4398.974358,4398.974358,4395.897435,4396.923076,4398.461538,4397.948717,4396.410256,4391.794871,4389.743589,4389.743589,4385.641025,4387.692307,4393.846153,4389.743589,4381.025641,...,4065.128205,4060.512820,4062.564102,4065.128205,4067.179487,4069.230769,4068.717948,4065.641025,4063.589743,4063.076923,4065.128205,4066.153846,4059.487179,4055.384615,4057.948717,4058.974358,4053.846153,4057.948717,4068.205128,4065.128205,4054.871794,4051.794871,4054.871794,4057.435897,4056.410256,4060.000000,4063.076923,4056.410256,4050.256410,4055.897435,4059.487179,4055.384615,4056.923076,4065.128205,4068.717948,4062.051282,4052.820512,4058.974358,4070.769230,4335.038295
1,4385.641025,4381.538461,4386.666666,4383.589743,4374.358974,4378.974358,4389.743589,4388.205128,4381.538461,4383.076923,4386.153846,4380.512820,4385.128205,4397.948717,4394.871794,4382.564102,4384.102564,4390.256410,4390.256410,4393.333333,4392.820512,4381.025641,4373.333333,4380.512820,4389.743589,4386.153846,4380.512820,4386.666666,4399.487179,4398.461538,4387.179487,4384.615384,4385.128205,4383.076923,4382.564102,4384.615384,4385.128205,4383.589743,4385.641025,4392.820512,...,4071.282051,4072.820512,4069.230769,4064.615384,4068.717948,4073.846153,4070.256410,4062.564102,4063.589743,4071.794871,4071.282051,4071.794871,4081.025641,4075.897435,4063.076923,4064.102564,4072.820512,4077.948717,4070.256410,4056.923076,4060.000000,4073.333333,4075.384615,4068.205128,4062.051282,4063.589743,4072.307692,4072.307692,4069.743589,4081.025641,4086.153846,4073.333333,4069.230769,4074.871794,4070.256410,4066.153846,4071.794871,4071.282051,4057.435897,4335.038295
2,4380.512820,4389.743589,4395.897435,4395.384615,4386.666666,4384.615384,4392.307692,4393.846153,4391.794871,4396.410256,4398.461538,4397.948717,4401.538461,4398.974358,4390.769230,4387.692307,4389.230769,4393.333333,4394.871794,4387.692307,4385.641025,4391.794871,4392.307692,4386.153846,4381.025641,4384.102564,4390.256410,4391.794871,4390.769230,4396.410256,4400.000000,4392.820512,4387.692307,4390.769230,4392.307692,4391.794871,4389.230769,4383.076923,4385.128205,4394.871794,...,4080.000000,4080.512820,4065.128205,4066.666666,4076.923076,4081.025641,4085.128205,4088.717948,4083.076923,4078.461538,4075.897435,4072.307692,4074.358974,4075.384615,4070.256410,4070.769230,4077.435897,4078.461538,4074.871794,4072.307692,4070.256410,4071.282051,4071.794871,4074.871794,4085.128205,4089.230769,4082.564102,4077.948717,4080.000000,4078.974358,4076.410256,4081.538461,4084.102564,4077.435897,4076.923076,4336.868564,4335.124777,4334.595502,4334.855340,4335.038295
3,4395.897435,4390.769230,4403.589743,4411.282051,4410.256410,4404.615384,4395.897435,4397.435897,4403.076923,4396.923076,4391.794871,4395.384615,4394.358974,4393.333333,4397.948717,4399.487179,4395.897435,4395.384615,4401.025641,4405.641025,4401.025641,4393.333333,4393.333333,4398.461538,4401.538461,4401.025641,4397.435897,4397.948717,4398.461538,4397.435897,4403.076923,4408.205128,4398.974358,4385.641025,4390.256410,4400.000000,4397.948717,4395.384615,4399.487179,4400.000000,...,4061.025641,4055.897435,4069.743589,4077.435897,4063.076923,4056.410256,4069.230769,4072.307692,4061.025641,4053.846153,4062.051282,4073.333333,4072.820512,4068.717948,4071.794871,4069.230769,4061.538461,4069.230769,4078.461538,4074.871794,4069.743589,4063.589743,4058.461538,4063.589743,4068.717948,4

In [0]:
pdf1=pd.read_csv('/content/drive/My Drive/BCI MNIST/yvar1.csv')
pdf2=pd.read_csv('/content/drive/My Drive/BCI MNIST/yvar2.csv')
pdf3=pd.read_csv('/content/drive/My Drive/BCI MNIST/yvar3.csv')

In [0]:
bigpred = pd.concat([pdf1, pdf2, pdf3], ignore_index=True, sort =False)


In [0]:
bigpred=bigpred['0'].map({'-1-1-1-1-1-1-1-1-1-1-1-1-1-1':10,'88888888888888':8,'66666666666666':6,'00000000000000': 0,'99999999999999':9,'55555555555555':5,'22222222222222':2,'11111111111111':1,'77777777777777':7,'33333333333333':3,'44444444444444':4,88888888888888:8,66666666666666:6,00000000000000: 0,99999999999999:9,55555555555555:5,22222222222222:2,11111111111111:1,77777777777777:7,33333333333333:3,44444444444444:4})

In [10]:
del pdf1
del pdf2
del pdf3
gc.collect()

20

In [11]:
len(bigpred.dropna())==len(bigpred)

True

In [12]:
bigpred.head()

0    6
1    7
2    9
3    9
4    0
Name: 0, dtype: int64

In [0]:
bigpred.drop(bigpred.columns[bigpred.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
#bigpred.head(100)

AttributeError: ignored

In [13]:
len(bigdata)==len(bigpred)

True

In [0]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_score, confusion_matrix, explained_variance_score, max_error, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge

#Generic function for making a classification model and accessing performance:
def classification_model(model, predictors, outcome):
  history=model.fit(predictors,outcome.values.ravel())
  
  predictions = model.predict(predictors)
  
  accuracy = metrics.accuracy_score(predictions,outcome)
  print('Accuracy : %s' % '{0:.3%}'.format(accuracy))

  kf = KFold(n_splits=2)
  error = []
  for train, test in kf.split(predictors):
    train_predictors = (predictors.iloc[train,:])
    
    train_target = outcome.iloc[train]
    
    model.fit(train_predictors, train_target.values.ravel())
    
    error.append(model.score(predictors.iloc[test,:], outcome.iloc[test]))
 
  print('Cross-Validation Score : %s' % '{0:.3%}'.format(np.mean(error)))

  return history

In [0]:
idx = np.random.permutation(df1.index)

,0
0,66666666666666
1,77777777777777
2,99999999999999
3,99999999999999
4,0


In [28]:
from sklearn.model_selection import train_test_split
df=bigdata
bigdata=(df-df.values.min())/(df.values.max()-df.values.min())
print('start')
X_train, X_test, y_train, y_test = train_test_split(bigdata, bigpred, test_size=0.20, random_state=42)
model = Ridge(alpha=1.0)
model.fit(X_train,y_train)
print('stop')

start
stop


In [0]:
#scaler = StandardScaler()

idx = np.random.permutation(X_test.index)
X_test=X_test.reindex(idx)
y_test=y_test.reindex(idx)
predictions = model.predict(X_test)
#prc=precision_score(predictions,y_test, average=None)
#cfm=confusion_matrix(predictions,y_test)
#accuracy = metrics.accuracy_score(predictions,y_test)
#false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, predictions)
#auc = auc(false_positive_rate, true_positive_rate)
#print(accuracy)
#print(prc)
#print(cfm)
#history=classification_model(model,X_test,y_test)

In [30]:
print(explained_variance_score(predictions,y_test))
print(max_error(predictions,y_test))
print(mean_absolute_error(predictions, y_test, multioutput='raw_values'))
print(mean_squared_error(predictions, y_test, multioutput='raw_values'))

-97.93416737942323
10.04630854405656
[2.50395031]
[8.33556872]
